In [7]:
import h5py
import numpy as np

In [8]:
f = h5py.File('my_model.h5', 'r')
list(f['model_weights'].keys())

['activation_5',
 'activation_6',
 'activation_7',
 'activation_8',
 'conv1d_1',
 'conv1d_2',
 'conv1d_3',
 'dense_2',
 'dropout_3',
 'dropout_4',
 'flatten_2',
 'max_pooling1d_4',
 'max_pooling1d_5',
 'max_pooling1d_6']

In [9]:
list(f['model_weights']['conv1d_1'].keys())

['conv1d_1_1']

In [10]:
list(f['model_weights']['conv1d_1']['conv1d_1_1'].keys())

['bias:0', 'kernel:0']

In [11]:
weights = {} # read weights from h5 file into new data structure
for key in f['model_weights'].keys():
    if len(list(f['model_weights'][key].keys())) > 0: # dont check empty fields
        
        key2 = list(f['model_weights'][key].keys())[0] # because of the way keras saves it
        
        weights [key] ={}
        for item in list(f['model_weights'][key][key2].keys()): # bias and kernel weights
            if 'bias' in item:
                weights[key]['bias'] = f['model_weights'][key][key2][item].value
            elif 'kernel' in item:
                weights[key]['kernel'] = f['model_weights'][key][key2][item].value

weights         

{'conv1d_1': {'bias': array([-0.03234073, -0.04306771, -0.00062039, -0.08663282,  0.06814551,
         -0.13665597,  0.10437986,  0.05147338, -0.04903092,  0.11343414,
          0.04661591,  0.0952212 , -0.07249567,  0.06355535, -0.00151818,
         -0.05318871, -0.13869078,  0.00323789, -0.07711463,  0.05038888,
          0.07027581, -0.01146772,  0.09931282,  0.13746944, -0.04002655,
         -0.13068414,  0.11402453,  0.03592147,  0.01777206, -0.01232621,
         -0.0669149 ,  0.05476604], dtype=float32),
  'kernel': array([[[ 0.0531412 , -0.02554819,  0.0006132 , ..., -0.12534058,
            0.01834949,  0.12942964],
          [ 0.06546013, -0.10227402, -0.04761713, ...,  0.16649884,
           -0.13860363,  0.05670927],
          [ 0.14166534, -0.07582323,  0.07630197, ...,  0.04448659,
           -0.01664439,  0.09898935],
          ...,
          [-0.03086024,  0.01396743, -0.11203548, ..., -0.03292698,
           -0.15231305,  0.11833104],
          [ 0.03976201, -0.08837412

In [12]:
w_range = {} # Calculate weight ranges to calculate shifts

for layer in weights.keys():
    w_range[layer]={}
    w_range[layer]['bias'] = {'max': np.max(np.abs(weights[layer]['bias'])), 
                              'min': np.min(np.abs(weights[layer]['bias'])), 
                              'mean': np.mean(np.abs(weights[layer]['bias']))}
    
    w_range[layer]['kernel'] = {'max': np.max(np.abs(weights[layer]['kernel'])), 
                                'min': np.min(np.abs(weights[layer]['kernel'])), 
                              'mean': np.mean(np.abs(weights[layer]['kernel']))}

w_range

{'conv1d_1': {'bias': {'max': 0.13869078,
   'mean': 0.06496157,
   'min': 0.0006203853},
  'kernel': {'max': 0.45838487, 'mean': 0.100128524, 'min': 7.8983285e-06}},
 'conv1d_2': {'bias': {'max': 0.14512946,
   'mean': 0.058545787,
   'min': 0.008338031},
  'kernel': {'max': 0.5595609, 'mean': 0.11296164, 'min': 4.3811993e-05}},
 'conv1d_3': {'bias': {'max': 0.14100273,
   'mean': 0.058709696,
   'min': 0.0022182853},
  'kernel': {'max': 0.58167064, 'mean': 0.13786913, 'min': 0.00014734392}},
 'dense_2': {'bias': {'max': 0.17303397,
   'mean': 0.08399361,
   'min': 0.029987605},
  'kernel': {'max': 0.7479633, 'mean': 0.15415601, 'min': 0.00023763049}}}

In [13]:
s_values = {'conv1d_1': {'bias': 9, 'kernel':8},
           'conv1d_2': {'bias': 9, 'kernel':7},
            'conv1d_3': {'bias': 9, 'kernel':7},
            'dense_1': {'bias': 9, 'kernel':8},
            'dense_2': {'bias': 9, 'kernel':7},
           } # shift values 
# because we are using a signed 8bit int, the sign bit is excluded: 7 bits for abs value of number. 
# Max values is 127 (+ or -)

In [14]:
w_new = {} # scale weights according to shift values

for layer in weights.keys():
    print('Layer: ', layer)
    print(weights[layer]['bias'].shape)
    print(weights[layer]['kernel'].shape)   
    w_new[layer]={}
    w_new[layer]['bias'] = np.ceil(weights[layer]['bias'] * pow(2, s_values[layer]['bias']) ).astype(int)
    w_new[layer]['kernel'] = np.ceil(weights[layer]['kernel'] * pow(2, s_values[layer]['kernel']) ).astype(int)
    print('b :: max:',np.max(np.abs(w_new[layer]['bias'])) , ' min: ', np.min(np.abs(w_new[layer]['bias'])))
    print('k :: max:',np.max(np.abs(w_new[layer]['kernel'])) , ' min: ', np.min(np.abs(w_new[layer]['kernel'])))

w_new

Layer:  conv1d_1
(32,)
(3, 45, 32)
b :: max: 71  min:  0
k :: max: 117  min:  0
Layer:  conv1d_3
(16,)
(3, 32, 16)
b :: max: 73  min:  2
k :: max: 75  min:  0
Layer:  dense_2
(10,)
(96, 10)
b :: max: 88  min:  16
k :: max: 95  min:  0
Layer:  conv1d_2
(32,)
(3, 32, 32)
b :: max: 74  min:  5
k :: max: 71  min:  0


{'conv1d_1': {'bias': array([-16, -22,   0, -44,  35, -69,  54,  27, -25,  59,  24,  49, -37,
          33,   0, -27, -71,   2, -39,  26,  36,  -5,  51,  71, -20, -66,
          59,  19,  10,  -6, -34,  29]),
  'kernel': array([[[ 14,  -6,   1, ..., -32,   5,  34],
          [ 17, -26, -12, ...,  43, -35,  15],
          [ 37, -19,  20, ...,  12,  -4,  26],
          ...,
          [ -7,   4, -28, ...,  -8, -38,  31],
          [ 11, -22, -13, ...,  -2,  32, -22],
          [-31, -23,  37, ..., -36,  14, -42]],
  
         [[  1, -32,  12, ..., -17, -25,  26],
          [-25,  10, -10, ..., -30,  21,  49],
          [-24,  12,   9, ...,  -3, -40,  33],
          ...,
          [-32, -32, -29, ...,  32,  -6,  17],
          [ 17,  45,  28, ..., -18, -26,  10],
          [-30, -22, -28, ...,  29,  -2,  15]],
  
         [[ 12, -45,  52, ...,   2,  29,   1],
          [ 23,  -9, -21, ...,  -6,  20,  21],
          [ -4, -43,  53, ..., -13, -25, -12],
          ...,
          [ 13,  46, -3

In [15]:
# creating weights file
w_file = open('arm_sign_nb_45/arm_sign_nb_weights.h', 'w')
w_file.write('/*Weights and shifts file*/\n')

conv_counter = 1
dense_counter = 1
name = None

print(' --- Writing Layer weights\n')

# Write weights and bias
layers = sorted(list(w_new.keys()))
for layer in layers:
    print(layer)
    if 'conv' in layer:
        name = 'CONV'+str(conv_counter)
        conv_counter +=1 
        
    elif 'dense' in layer:
        name = 'IP'+str(dense_counter)
        dense_counter +=1
        
    if name:   
        w_file.write('#define '+name+'_WT {')
        w_new[layer]['kernel'].tofile(w_file, sep=',', format='%d')
        w_file.write('}\n')
        
        w_file.write('#define '+name+'_BIAS {')
        w_new[layer]['bias'].tofile(w_file, sep=',', format='%d')
        w_file.write('}\n\n')

w_file.write('\n')

# Write Shifts
print(' --- Writing Layer shifts\n')
conv_counter = 1
dense_counter = 1
name = None

layers = sorted(list(s_values.keys()))
for layer in layers:
    print(layer)
    
    if 'conv' in layer:
        name = 'CONV'+str(conv_counter)
        conv_counter +=1
    elif 'dense' in layer:
        name = 'IP'+str(dense_counter)
        dense_counter +=1
        
    if name:   
        w_file.write('#define '+name+'_OUT_RSHIFT ')
        w_file.write('{:d}'.format(s_values[layer]['kernel']))
        w_file.write('\n')
        
        w_file.write('#define '+name+'_BIAS_LSHIFT ')
        w_file.write('{:d}'.format(s_values[layer]['bias']))
        w_file.write('\n\n')

# wRITE input shifts
print(' --- Writing Input shifts\n')
w_file.write('#define INPUT_MEAN_SHIFT '+ str(174)+ '\n') # from analysis of dataset
w_file.write('#define INPUT_RIGHT_SHIFT '+ str(8)) # each value is up to 255 (2^8)

w_file.close()

 --- Writing Layer weights

conv1d_1
conv1d_2
conv1d_3
dense_2
 --- Writing Layer shifts

conv1d_1
conv1d_2
conv1d_3
dense_1
dense_2
 --- Writing Input shifts

